<a href="https://colab.research.google.com/github/suwatoh/Python-learning/blob/main/118_%E9%AB%98%E6%B0%B4%E6%BA%96%E3%81%AE%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB%E6%93%8D%E4%BD%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

高水準のファイル操作
====================

一時ファイル
------------

**一時ファイル**（temporary file）とは、ソフトウェアが作業中のデータを一時的に保管するために作成するファイルのことをいう。

一時ファイルは、安全上、ファイル作成権限があり、かつ、他のユーザーから隔離されたディレクトリに作成する必要がある。このため、プラットフォームは専用のディレクトリを用意している。

  * Windows: `C:\Users\<ユーザー名>\AppData\Local\Temp` （または環境変数 `TEMP` か `TMP` が指定する場所）
  * macOS: `/var/folders/...` の下にランダムな名前で作られるフォルダ （または環境変数 `TMPDIR` が指定する場所）
  * Linux: `/tmp` または `/var/tmp`

各ソフトウェアが専用のディレクトリに一時ファイルを作成する場合、ファイル名の衝突に注意する必要がある。

標準ライブラリの `tempfile` モジュールは、プラットフォームの違いやファイル名を意識することなく安全に一時ファイルを作成する機能を提供する。

### TemporaryFile

``` python
tempfile.TemporaryFile(mode='w+b', buffering=-1, encoding=None, newline=None, suffix=None, prefix=None, dir=None, *, errors=None)
```

この関数は、一時ファイルを作成し、ファイルオブジェクトを返す。ファイルは、可能な限り最も安全な手段で作成される。すなわち、ファイル名が既存のファイルと競合しないことが保証され、また、作成したユーザーのみファイルを読み書き可能なパーミッション設定が行われる。ファイルオブジェクトが閉じられると、即座にファイルが自動的に削除される。

一般のファイルオブジェクトと同じで、この関数が返すファイルオブジェクトもコンテキストマネージャーなので、with 文の中で使用することができる。

引数 `mode`、`buffering`、`encoding`、`newline`、`errors` は、内部で呼び出す `open()` 関数に対する引数として解釈される。残りの引数については次のとおり。

| 引数 | 意味 |
|:---|:---|
| `suffix` | `None` でない場合、ファイル名はその接尾辞で終わる |
| `prefix` | `None` でない場合、ファイル名はその接頭辞で始まる |
| `dir` | `None` でない場合、ファイルはそのディレクトリ下に作成される。`None` の場合、`TMPDIR`、`TEMP`、または `TMP` 環境変数の値が使われる |

In [ ]:
import tempfile

file_name = ""
with tempfile.TemporaryFile() as f:
    file_name = f.name
    f.write(b"abcd")  # デフォルトではバイナリモード
    f.seek(0)
    print(f"{f.read()=}")

print(f"{file_name=}")  # Unix 系ではファイル名が不可視。Windows ではファイルパスを得る

f.read()=b'abcd'
file_name=47


### NamedTemporaryFile

``` python
tempfile.NamedTemporaryFile(mode='w+b', buffering=-1, encoding=None, newline=None, suffix=None, prefix=None, dir=None, delete=True,
                            *, errors=None, delete_on_close=True)
```

この関数も、一時ファイルを作成し、ファイルオブジェクトを返すが、以下の点で `tempfile.TemporaryFile()` と異なる。

  * ファイルオブジェクトの `name` 属性で一時ファイルのパスが得られる。
  * `delete` 引数が追加され、これが`False` の場合、ファイルオブジェクトを閉じてもファイルが自動的には削除されない。デフォルトは `True`。
  * Python 3.12 で `delete_on_close` キーワード専用引数が追加され、`delete` が `True` で、`delete_on_close` が `False` の場合、with 文で作成された一時ファイルは with 文が終わったときにのみ削除され、途中でファイルオブジェクトが閉じられてもファイルは削除されないようになった。デフォルトは `True`。

In [ ]:
import tempfile
from pathlib import Path

path = None
with tempfile.NamedTemporaryFile(delete=False) as f:
    f.write(b'abcd')
    f.seek(0)
    print(f.read())
    path = f.name
    print(f"{path=}")
    assert Path(path).exists()

assert Path(path).exists()
Path(path).unlink()
assert not Path(path).exists()

b'abcd'
path='/tmp/tmpy4iakif_'


### SpooledTemporaryFile

``` python
tempfile.SpooledTemporaryFile(max_size=0, mode='w+b', buffering=-1, encoding=None, newline=None, suffix=None, prefix=None, dir=None,
                              *, errors=None)
```

これは、`SpooledTemporaryFile` クラスのコンストラクタである。 `SpooledTemporaryFile` クラスは、データを基本的にメモリ上に書き出す。ファイルサイズが `max_size`（バイト数）を超えるか、ファイルの `fileno()` メソッドが呼び出されるとディスク上に書き出すため、メモリ消費量が増えすぎるということがない。ディスクに書き出したあとの動作は `TemporaryFile()` と同様である。

In [ ]:
import tempfile

# 最大サイズを 1MB で指定
with tempfile.SpooledTemporaryFile(max_size=1024*1024, mode='w+t') as tmp:
    # 書き込み
    tmp.write("Hello, world!\n")
    tmp.write("これは一時ファイルです。\n")
    # 読み込み
    tmp.seek(0)
    for line in tmp:
        print(line.strip())

Hello, world!
これは一時ファイルです。


`SpooledTemporaryFile` オブジェクトは `rollover()` メソッドを持つ。`rollover()` メソッドを呼ぶと、サイズに関係なく即座にディスク上の一時ファイルに移行する。

In [ ]:
import tempfile
from pathlib import Path

with tempfile.SpooledTemporaryFile(max_size=1024, mode='w+t') as tmp:
    tmp.write("small data")
    tmp.rollover()  # ディスクへ移行
    tmp.seek(0)
    print(tmp.read())

small data


### TemporaryDirectory

``` python
tempfile.TemporaryDirectory(suffix=None, prefix=None, dir=None, ignore_cleanup_errors=False, *, delete=True)
```

これは、`TemporaryDirectory` クラスのコンストラクタである。`TemporaryDirectory` クラスは、一時的に使用する作業用のディレクトリの作成をサポートする。

| 引数 | 意味 |
|:---|:---|
| `suffix` | `None` でない場合、ディレクトリ名はその接尾辞で終わる |
| `prefix` | `None` でない場合、ディレクトリ名はその接頭辞で始まる |
| `dir` | `None` でない場合、ディレクトリは `dir` のサブディレクトリとして作成される。`None` の場合、`TMPDIR`、`TEMP`、または `TMP` 環境<br /><br />変数の値が使われる |
| `ignore_cleanup_errors` | `True` の場合、ディレクトリの削除中のエラーを無視する |

`TemporaryDirectory` の属性:

| 属性 | 意味 |
|:---|:---|
| `name` | ディレクトリのパスを表す文字列を格納する属性 |

`TemporaryDirectory` のメソッド:

| メソッド | 機能 | 戻り値 |
|:---|:---|:--:|
| `cleanup()` | ディレクトリを中身のファイルごと削除する。`ignore_cleanup_errors` が`True` の場合、ファイルの削除中にエラーが発生しても<br /><br />残りのファイルの削除を継続する | `None` |

`TemporaryDirectory` は、コンテキストマネージャーの機能をサポートしていて、`__enter__()` は `name` 属性を返し、`__exit__()` は `cleanup()` を呼び出す。ただし、Python 3.12 以降では、コンストラクタに `delete` キーワード専用引数が追加され、これに `False` を与えてインスタンス化した場合、`__exit__()` は `cleanup()` を呼ばず、何もしない（つまり with のブロックを抜けてもディレクトリはそのまま残る）。

In [ ]:
import tempfile
from pathlib import Path

path = None
with tempfile.TemporaryDirectory() as tmpdirname:
    print(f"{tmpdirname=}")
    p = Path(tmpdirname, "test.txt")
    p.touch()  # ファイルを作成
    print(f"{[x.name for x in Path(tmpdirname).iterdir()]}")

assert not Path(tmpdirname).exists()

tmpdirname='/tmp/tmp2qfosf33'
['test.txt']


### 低レベル関数

`tempfile.TemporaryFile` などの内部で呼び出されている低レベル関数を直接使用することもできる。

``` python
tempfile.mkstemp(suffix=None, prefix=None, dir=None, text=False)
```

可能な限り最も安全な手段で一時ファイルを生成する。`tempfile.TemporaryFile()` と違って、`tempfile.mkstemp()` で作成された一時ファイルは、用済みになった時に手動で削除する必要がある（あるいは、システムによって一時ファイルが自動的に削除されることを期待することになる）。

引数の意味とデフォルトは、`tempfile.TemporaryFile()` のものと同じ。

``` python
tempfile.mkdtemp(suffix=None, prefix=None, dir=None)
```

可能な限り安全な方法で一時ディレクトリを作成する。`tempfile.TemporaryDirectory()` と違って、`tempfile.mkdtemp()` で作成された一時ディレクトリは、用済みになった時に手動で削除する必要がある（あるいは、システムによって一時ディレクトリが自動的に削除されることを期待することになる）。

引数の意味とデフォルトは、`tempfile.TemporaryDirectory()` のものと同じ。

``` python
tempfile.gettempdir()
```

一時ファイルに用いられるディレクトリの名前を返す。

この関数はモジュール内の全ての関数の `dir` 引数のデフォルト値を定義する。具体的には、以下の順番でディレクトリを検索する。

  1. 環境変数 `TMPDIR` で与えられているディレクトリ名。
  2. 環境変数 `TEMP` で与えられているディレクトリ名。
  3. 環境変数 `TMP` で与えられているディレクトリ名。
  4. プラットフォーム依存の場所。
  5. 現在の作業ディレクトリ。

この検索の結果はキャッシュされる。

In [ ]:
import tempfile
tempfile.gettempdir()

'/tmp'

コピーと移動
------------

ファイルやディレクトリに対するコピーや移動といった高水準の操作には、標準ライブラリの `shutil` モジュールを使う。

### ファイルのコピー

``` python
shutil.copyfileobj(fsrc, fdst[, length])
```

この関数は、ファイルオブジェクト `fsrc` の内容をファイルオブジェクト `fdst` にコピーする。戻り値は `None`。実装としては、`fsrc` の `read()` メソッドによる読み込みと `fdst` の `write()` メソッドによる書き込みの反復を、`length`（ `length` が与えられていない場合は `shutil.COPY_BUFSIZE`）サイズ単位で行っている。このため、ファイルのパーミッション（権限）とメタデータ（最終アクセス時間、最終変更時間など）はコピーしない。

`fsrc` と `fdst` には、`io.BytesIO` や `io.StringIO` も使える。

In [ ]:
import io
import shutil
src = io.StringIO("あいう")
dest_bin = io.BytesIO()
dest = io.TextIOWrapper(dest_bin, encoding="utf-8")
shutil.copyfileobj(src, dest)
dest.flush()
dest_bin.getvalue()

b'\xe3\x81\x82\xe3\x81\x84\xe3\x81\x86'

``` python
shutil.copyfile(src, dst, *, follow_symlinks=True)
```

この関数は、path-like オブジェクト `src` の内容を path-like オブジェクト `dst` にコピーし、`dst` を返す。実装としては、`src` を `open()` で開いて、返されたファイルオブジェクトを `copyfileobj()` に渡している。このため、ファイルのパーミッションとメタデータはコピーしない。

`dst` はファイルでなければならない。`dst` が既に存在するファイルを指定している場合、それは置き換えられる。`dst` が存在しない中間ディレクトリを含むとエラーが発生する。あらかじめ直上のディレクトリまで作成しておく必要がある。このような仕様は、実装が `open(src, 'rb')` と `open(dst, 'wb')` を呼び出していることによる。

`src` と `dst` が同じファイルだった場合、`shutil.SameFileError` 例外が発生する。

`src` がシンボリックリンクの場合、キーワード専用引数 `follow_symlinks` が偽なら `dst` はシンボリックリンクとして作成され、`follow_symlinks` が真なら `dst` には `src` のリンク先のファイルがコピーされる。

``` python
shutil.copymode(src, dst, *, follow_symlinks=True)
```

この関数は、パーミッションを `src` から `dst` にコピーする。最終アクセス時間、最終変更時間などのメタデータは一切コピーしない。戻り値は `None`。引数は、`shutil.copyfile()` と同じ。

``` python
shutil.copystat(src, dst, *, follow_symlinks=True)
```

この関数は、パーミッションを `src` から `dst` にコピーし、さらに最終アクセス時間、最終変更時間などの一部メタデータもコピーする。戻り値は `None`。引数は、`shutil.copyfile()` と同じ。

以下のメタデータはコピーされないことに注意する。

  * POSIX プラットフォーム: ACL、ファイルのオーナー、グループ
  * macOS: リソースフォーク（ファイルタイプ、クリエイター、アイコン、ウィンドウ位置、バージョン情報など）
  * Windows: 所有者、ACL、代替データストリーム（ADS）など

``` python
shutil.copy(src, dst, *, follow_symlinks=True)
```

この関数は、ファイルの内容とパーミッションを `src` から `dst` にコピーし、`dst` を返す。関数の内部では、 `shutil.copyfile()` と `shutil.copymode()` を呼び出している。ただし、`dst` 引数はそのままこれらの関数に渡されるのではなく、`dst` がディレクトリなら `dst` のパスと `src` のベース（パス要素の末尾）を連結した文字列に加工された上で渡される。したがって、**この関数は、ファイルをディレクトリにコピーすることができる**。

``` python
shutil.copy2(src, dst, *, follow_symlinks=True)
```

この関数は、ファイルのメタデータを保持しようとすることを除けば `copy()` と等価である。関数の内部では、`shutil.copyfile()` と `shutil.copystat()` を呼び出している。したがって、全てのメタデータがコピーされるわけではないことに注意する。

### ディレクトリのコピー

``` python
shutil.copytree(src, dst, symlinks=False, ignore=None, copy_function=copy2, ignore_dangling_symlinks=False, dirs_exist_ok=False)
```

この関数は、ディレクトリツリーを丸ごとコピーする。つまり、ディレクトリを配下のファイルやサブディレクトリも含めて再帰的にコピーする。戻り値は、`dst`。

| 引数 | 意味 |
|:---|:---|
| `src` | コピー対象のディレクトリを指定する。path-like オブジェクトが使える |
| `dst` | コピー後のディレクトリを指定する。path-like オブジェクトが使える |
| `symlinks` | `True` の場合、シンボリックリンクはコピー後もシンボリックリンクとなる。`False` の場合、リンク先のファイルそのものがコピーさ<br /><br />れる |
| `ignore` | 呼び出し可能オブジェクトを指定すると、そのオブジェクトが返す相対パス（カレントディレクトリを基準とする）の集合に含まれる<br /><br />ディレクトリやファイルをコピーしない。呼び出し可能オブジェクトは 2 つの引数を取るものでなければならず、第 1 引数にはディ<br /><br />レクトリ、第 2 引数にはその中身のリストが渡される |
| `copy_function` | コピーに用いる関数を指定する。デフォルトでは `shutil.copy2()` が使用される |
| `ignore_dangling_symlinks` | `True` を指定すると、引数 `symlinks` が `False` のときにリンク先が存在しない場合でもエラーを出さないようになる |
| `dirs_exist_ok` | `False`（デフォルト）の場合、`dst` が既存のディレクトリならエラーが発生する。`True` を指定すると、`dst` が既存のディレクトリ<br /><br />でもエラーにならない |

`ignore` 引数に渡す呼び出し可能オブジェクトは、次のユーティリティ関数で簡単に作成することができる。

``` python
shutil.ignore_patterns(*patterns)
```

このファクトリー関数は、 `shutil.copytree()` 関数の `ignore` 引数に渡すための呼び出し可能オブジェクトを作成する。`patterns` には glob パターンを複数指定でき、指定したパターンにマッチするファイル名およびディレクトリ名を含む相対パスの集合を返す呼び出し可能オブジェクトを作成できる。

以下のコードは、Colab が用意する sample_data ディレクトリを一時ディレクトリにコピーする。ただし、`shutil.ignore_patterns()` を使用して、拡張子 `.md` および `.json` のファイルをコピー対象から除外している。

In [ ]:
import tempfile
import shutil
from pathlib import Path

ignore = shutil.ignore_patterns("*.md", "*.json")
src = Path("./sample_data")
print(f"{ignore(src, [x.name for x in src.iterdir()])=}")

with tempfile.TemporaryDirectory() as td:
    dest = shutil.copytree(src, Path(td) / Path("sample_data"), ignore=ignore)
    print(f"{dest=}")
    print(f"{[x.name for x in Path(dest).iterdir()]}")

ignore(src, [x.name for x in src.iterdir()])={'README.md', 'anscombe.json'}
dest=PosixPath('/tmp/tmpp2k1s8co/sample_data')
['california_housing_train.csv', 'mnist_test.csv', 'mnist_train_small.csv', 'california_housing_test.csv']


### ファイルとディレクトリの移動

``` python
shutil.move(src, dst, copy_function=copy2)
```

この関数は、ファイルまたはディレクトリ `src` を再帰的に別の場所 `dst` に移動して、`dst` を返す。`src` および `dst` は path-like オブジェクトが使える。`copy_function` にコピーに用いる関数を指定する。デフォルトでは shutil.copy2() が使用される。

`dst` が存在するディレクトリの場合、 `src` はそのディレクトリの中へ移動する。`dst` が存在していてそれがディレクトリでない場合、内部で `os.rename()` を使用するため、`os.rename()` と同じ結果になる。

### ディレクトリの削除

``` python
shutil.rmtree(path, ignore_errors=False, onerror=None, *, onexc=None)
```

この関数は、ディレクトリを中身のファイルやサブディレクトリごと削除する。

| 引数 | 意味 |
|:---|:---|
| `path` | 削除するディレクトリを path-like オブジェクトで指定する |
| `ignore_errors` | `True` の場合、削除に失敗したことによるエラーは無視される。`False`（デフォルト）の場合、これらのエラーは `onerror` または `onexc` に指<br /><br />定した呼び出し可能オブジェクトで処理され、両方が省略された場合は例外を送出する |
| `onerror` | `function`, `path`, `excinfo` の 3 つの引数をとる呼び出し可能オブジェクトを指定する。`function` には例外が発生した関数が、`path` には<br /><br /> `function` に渡されたパスが、`excinfo` には `sys.exc_info()` の戻り値が渡される。この引数は Python 3.12 で非推奨とされた |
| `onexc` | キーワード専用。Python 3.12 で追加。受け取る 3 番目の引数が発生した例外であることを除いて `onerror` と同様である |

Python 3.13 からは、`rmtree()` は、トップレベルのパスを除き、`FileNotFoundError` 例外を無視するようになった。また、`OSError` およびそのサブクラス以外の例外は、常に呼び出し元へ伝播されるようになった。

### パスオブジェクトのメソッド

Python 3.14 で、`pathlib.Path` にファイルのコピー・移動を行うメソッドが追加された。

``` python
Path.copy(target, *, follow_symlinks=True, preserve_metadata=False)
```

``` python
Path.copy_into(target_dir, *, follow_symlinks=True, preserve_metadata=False)
```

``` python
Path.move(target)
```

``` python
Path.move_into(target_dir)
```

圧縮とアーカイブ
----------------

### GZIP

**GZIP**（ジー・ジップ）は、GNU プロジェクトによって開発されたデータの圧縮・展開ツール、およびその圧縮データのフォーマットである。ツール（gzip および gunzip）は、多くの UNIX／UNIX 系 OS に標準搭載されている。圧縮後のファイル名は、元のファイル名に .gz 拡張子を付加することが多い。

データ圧縮には、**Deflate** と呼ばれる、次の 2 つの技術を組み合わせたアルゴリズムが使用される。

  1. **Lempel-Ziv アルゴリズム（LZ77）**:
      * 過去に出現したデータのパターン（文字列）を「辞書」として保持する（辞書式圧縮）。
      * 同じパターンが再び現れたとき、「〇文字前にあった〇文字と同じ」という「位置と長さの情報（ポインタ）」に置き換えることでデータを短縮する。  
      例：「[寿司屋]の[寿司]がおいしい」 → 「[寿司屋]の[1文字前]の[2文字]がおいしい」
  2. **ハフマン符号化（エントロピー符号化）**:
      * データの中で頻繁に出現する文字や記号に短いコード（符号）を割り当てる。
      * 出現頻度の低い文字には長いコードを割り当てる。
      * これにより、全体として必要なデータ量を削減できる。

In [2]:
# Deflate アルゴリズムについては、次の動画が参考になる
from IPython.display import HTML
HTML('''<iframe width="640" height="360" src="https://www.youtube.com/embed/rXwiOOoDf8c"
  frameborder="0" allow="accelerometer; clipboard-write; encrypted-media; gyroscope; picture-in-picture"
  referrerpolicy="strict-origin-when-cross-origin"></iframe>''')

このアルゴリズムでは任意の形式のデータを欠損なく圧縮（ロスレス圧縮）できるため、完全に元に戻すこと（展開）が可能である。一方、テキストデータを前提に設計されているため、写真や動画などのバイナリデータ（すでに効率的に符号化され、繰り返しパターンが少ないデータ）に対しては、圧縮効果は限定的となる。

データフォーマットは、次の図のようになっている。

``` text
┌──────────┬─────────────┬────────────┐
│ヘッダー部 (Header) │ 圧縮データ本体 (Payload) │ トレーラー部 (Trailer) │
└──────────┴─────────────┴────────────┘
```

  * **ヘッダー部**（基本的には 10 バイト）:  
  先頭の 3 バイトのマジックナンバー `0x1F8B08` のほか、タイムスタンプや OS 種別などのメタデータが含まれる。
  * **トレーラー部**（末尾の 8 バイト）:  
  データが正しく転送・解凍されたかを確認するための**チェックサム**（CRC32）と、元のファイルのサイズが記録されている。

標準ライブラリの `gzip` モジュールは、GZIP 圧縮、展開を行うシンプルなインターフェースを提供している。

``` python
gzip.open(filename, mode='rb', compresslevel=9, encoding=None, errors=None, newline=None)
```

この関数は、GZIP 圧縮ファイルをバイナリまたはテキストモードで開き、ファイルオブジェクトを返す。

| 引数 | 意味 |
|:---|:---|
| `filename` | ファイル名（文字列またはバイト列）か、既存のファイルオブジェクト |
| `mode` | バイナリモード用に `'r'`、`'rb'`、`'a'`、`'ab'`、`'w'`、`'wb'`、`'x'`、`'xb'`、テキストモード用に `'rt'`、`'at'`、`'wt'`、`'xt'`<br /><br /> を指定可能。デフォルトは `'rb'` |
| `compresslevel` | 圧縮レベルを 0 から 9 までの整数で指定する。0 は非圧縮、1 は最も高速で最小限の圧縮、9 は最も低速で最大限の圧縮。デ<br /><br />フォルトは 9 |
| `encoding`, `errors`, `newline` | `open()` 関数と同様。バイナリモードでは指定できない |

圧縮および展開時にエラーが発生した場合、`zlib.error` 例外が送出される。

GZIP 圧縮されたファイルを作成し、読み込む例:

In [ ]:
import gzip

# 圧縮されたファイルを作成
content = b"Lots of content here"
with gzip.open('file.txt.gz', 'wb') as f:
    f.write(content)

# 圧縮されたファイルを読み込む
with gzip.open('file.txt.gz', 'rb') as f:
    file_content = f.read()
assert file_content == content

既存のファイルを GZIP 圧縮する例:

In [ ]:
import os
import gzip
import shutil

# 既存のファイルのサイズ（バイト）を表示
print('圧縮前:', os.path.getsize('/content/sample_data/mnist_test.csv'))

# GZIP 圧縮
with open('/content/sample_data/mnist_test.csv', 'rb') as f_in:
    with gzip.open('/mnist_test.csv.gz', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

# 圧縮ファイルのサイズ（バイト）を表示
print('圧縮後:', os.path.getsize('/mnist_test.csv.gz'))

圧縮前: 18289443
圧縮後: 2112696


メモリ上で GZIP 圧縮と展開を行うための関数も提供されている。

``` python
gzip.compress(data, compresslevel=9, *, mtime=0)
```

この関数は、第 1 引数の `bytes` オブジェクトを圧縮し、圧縮済みデータを含む `bytes` オブジェクトを返す。

キーワード専用の `mtime` 引数は、ヘッダーのタイムスタンプ（最終更新日時）を UNIX 時間で指定する。`None` を指定すると現在時刻が使用される。デフォルト値は Python のバージョンによって異なるので注意。

``` python
gzip.decompress(data)
```

この関数は、GZIP 圧縮された `bytes` オブジェクトを展開し、展開後のデータを含む `bytes` オブジェクトを返す。複数メンバーの GZIP データ（複数の GZIP ブロックを連結したもの）も展開できる。

In [2]:
import gzip
s_in = b"Lots of content here"
s_out = gzip.compress(s_in)
print(s_out)
assert gzip.decompress(s_out) == s_in

b'\x1f\x8b\x08\x00Ij;i\x02\xff\xf3\xc9/)V\xc8OSH\xce\xcf+I\xcd+Q\xc8H-J\x05\x00~u \x8e\x14\x00\x00\x00'


### BZIP2

**BZIP2**（ビージップツー）は、GZIP よりも後発のデータの圧縮・展開ツールであり、およびその圧縮データのフォーマットである。圧縮後のファイル名は、元のファイル名に .bz2 拡張子を付加することが多い。

データ圧縮には、ブロックソートテキスト圧縮アルゴリズムとハフマン符号化を組み合わせたアルゴリズムが使用される。LZ77 と比較すると、圧縮率は高いが、圧縮速度は劣っている。

データフォーマットは、先頭の 3 バイトのマジックナンバー `BZh`（`0x455A68`）とブロックサイズを含むヘッダー、ブロックに分割された圧縮データ本体、最後に CRC32 チェックサムが含まれるトレーラーで構成される。

標準ライブラリの `bz2` モジュールは、BZIP2 圧縮、展開を行うシンプルなインターフェースを提供している。

``` python
bz2.open(filename, mode='rb', compresslevel=9, encoding=None, errors=None, newline=None)
```

この関数は、`gzip.open()` と同様に使うことができ、BZIP2 圧縮ファイルをバイナリまたはテキストモードで開き、ファイルオブジェクトを返す。

In [ ]:
import os
import bz2
import shutil

# 既存のファイルのサイズ（バイト）を表示
print('圧縮前:', os.path.getsize('/content/sample_data/mnist_test.csv'))

# BZIP2 圧縮
with open('/content/sample_data/mnist_test.csv', 'rb') as f_in:
    with bz2.open('mnist_test.csv.bz2', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

# 圧縮ファイルのサイズ（バイト）を表示
print('圧縮後:', os.path.getsize('mnist_test.csv.bz2'))

圧縮前: 18289443
圧縮後: 1590041


### LZMA

**LZMA**（Lempel-Ziv-Markov chain-Algorithm）は、LZ77 を改良した辞書式圧縮を使用するデータ圧縮アルゴリズムである。GZIP、BZIP2 と比較すると、圧縮時にはより多くの時間とメモリを消費するが、圧縮率では概ね優位で、また展開速度は GZIP より多少遅いが BZIP2 よりは速い傾向にある。

**LZMA2** は、LZMA を内包するコンテナ形式であり、複数の異なる圧縮設定の LZMA データを含むことができる。圧縮しにくいデータは非圧縮のまま格納できる。現在は LZMA 単体で圧縮することは少なく、LZMA2 が主流である。

LZMA/LZMA2 は、Windows を主な対応 OS とする 7-Zip の主要な圧縮アルゴリズムとして開発され、LZMA SDK がパブリックドメインとして公開されている。

Unix 系 OS では、XZ Utils が LZMA を組み込んだファイルフォーマット（拡張子 .lzma）と、LZMA2 を組み込んだファイルフォーマット（拡張子 .xz）を公開し、とくに .xz フォーマットが普及している。

標準ライブラリの `lzma` モジュールは、LZMA 圧縮、展開を行うインターフェースを提供している。3 種類のコンテナフォーマットに対応しており、モジュール定数で使い分けできるようにしている。

| モジュール定数 | フォーマット |
|:---|:---|
| `lzma.FORMAT_XZ` | .xz ファイルフォーマット |
| `lzma.FORMAT_ALONE` | .lzma ファイルフォーマット |
| `lzma.FORMAT_RAW` | 特定のフォーマットを使わない、LZMA 圧縮データそのもの |

``` python
lzma.open(filename, mode='rb', *, format=None, check=-1, preset=None, filters=None, encoding=None, errors=None, newline=None)
```

この関数は、`gzip.open()` と同様に使うことができ、LZMA 圧縮ファイルをバイナリまたはテキストモードで開き、ファイルオブジェクトを返す。

`format`、`check`、`preset`、`filters` のオプションは、読み込み用にファイルを開いた場合と、書き出し用に開いた場合とで意味が異なる。

読み込み用に開いた場合:

| 引数 | 意味 |
|:---|:---|
| `format` | コンテナフォーマットをモジュール定数で指定する。デフォルトは `lzma.FORMAT_AUTO` で、LZMA と LZMA2 圧縮を展開できる |
| `check` | 使用できない |
| `preset` | 使用できない |
| `filters` | 展開設定を指定する「フィルタチェイン」を指定する |

書き出し用に開いた場合:

| 引数 | 意味 |
|:---|:---|
| `format` | コンテナフォーマットをモジュール定数で指定する。デフォルトは `lzma.FORMAT_XZ` |
| `check` | 圧縮データに組み込むチェックサムをモジュール定数で指定する。`lzma.FORMAT_XZ` のデフォルトは `CHECK_CRC64`（CRC64）。`lzma.FORMAT_ALONE`<br /><br /> と `lzma.FORMAT_RAW` のデフォルトは `CHECK_NONE`（チェックサムなし） |
| `preset` | 0 から 9 までの整数値。高い数値を指定すると圧縮率が上がるが、圧縮にかかる時間が長くなる |
| `filters` | `preset` を指定しない場合に、圧縮設定を指定する「フィルタチェイン」を指定する。`preset` も `filters` も指定されなかった場合は `preset=6` と<br /><br />同等の圧縮設定が使用される |

「フィルタチェイン」の形式については、公式ドキュメントの「[カスタムフィルタチェインの指定](https://docs.python.org/ja/3/library/lzma.html#specifying-custom-filter-chains)」に説明があるが、設定の説明がほとんどないため効果は不明。

デフォルトの圧縮設定を変更しても、圧縮率はそれほど上がらず、処理時間やメモリ消費量が大きく増えるだけなので、公式ドキュメントもデフォルトのままで使用することを推奨している。

圧縮および展開時にエラーが発生した場合、`lzma.LZMAError` 例外が送出される。

In [ ]:
import os
import lzma
import shutil

# 既存のファイルのサイズ（バイト）を表示
print('圧縮前:', os.path.getsize('/content/sample_data/mnist_test.csv'))

# LZMA 圧縮
with open('/content/sample_data/mnist_test.csv', 'rb') as f_in:
    with lzma.open('mnist_test.csv.xz', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

# 圧縮ファイルのサイズ（バイト）を表示
print('圧縮後:', os.path.getsize('mnist_test.csv.xz'))

圧縮前: 18289443
圧縮後: 1674996


### Zstandard

**Zstandard**（ズィースタンダード）は、Meta 社の zstd ユーティリティに実装されたロスレスのデータ圧縮アルゴリズムである。LZ77 と、高度に最適化されたエントロピー符号化・復号を併用している。圧縮レベルを -1 から 22 まで指定でき、レベルを上げるほど圧縮率が向上する。デフォルトの圧縮レベル 3 では、Deflate アルゴリズムとほぼ同等の圧縮率で、より高速な圧縮・展開を実現するよう設計されている。現在は仕様が公開され、国際標準化団体 IETF によって規格化されている。高速性と効率性の高さから、今後はリアルタイム圧縮処理の分野で主流になるものと予想される。

標準ライブラリの `compression.zstd` モジュールは、Zstandard を用いてデータを圧縮、展開するための関数などを提供している。Python 3.14 で追加された。

``` python
compression.zstd.open(file, /, mode='rb', *, level=None, options=None, zstd_dict=None, encoding=None, errors=None, newline=None)
```

この関数は、`gzip.open()` と同様に使うことができ、Zstandard 圧縮ファイルをバイナリまたはテキストモードで開き、ファイルオブジェクトを返す。

圧縮および展開時にエラーが発生した場合、`compression.zstd.ZstdError` 例外が送出される。

メモリ上で GZIP 圧縮と展開を行うための関数も提供されている。

``` python
compression.zstd.compress(data, level=None, options=None, zstd_dict=None)
```

``` python
compression.zstd.decompress(data, zstd_dict=None, options=None)
```

### TAR

複数のファイルを 1 つのファイルにまとめることを**アーカイブ**（archive）という。

**TAR**（ター）は、UNIX で標準のアーカイブファイルフォーマットである。TAR を処理する tar コマンドは UNIX／UNIX 系 OS に標準搭載されている。アーカイブ後のファイル名は、元のファイル名に .tar 拡張子を付加することが多い。

TAR は圧縮をサポートしていない。複数のファイルをまとめて 1 つの圧縮ファイルを作成するには、TAR と GZIP または BZIP2、LZMA（XZ Utils）、Zstandard を組み合わせる。ファイルの拡張子は、それぞれ .tar.gz、.tar.bz2、.tar.xz、tar.zst となる。それぞれ略して .tgz、.tbz2、.txz、.tzst とされることも多い。

TAR のフォーマットは、以下のようにヘッダーとデータが交互に存在する。ヘッダーのサイズは 512 バイト。データ領域は 512 バイトの倍数になるように 0 で埋められる（パディング）。

``` text
┌──────┬───────┬──────┬───────┬───
│ヘッダー１  │ データ１     │ ヘッダー２ │ データ２     │ ・・・
└──────┴───────┴──────┴───────┴───
```

ヘッダーには、ファイルの名前、サイズ、パーミッション、所有者／グループ情報、最終更新日時、チェックサムなどのメタデータが含まれ、UNIX 系ファイルシステムのあらゆる詳細を保存できるように設計されている。この機能は強力で、TAR ファイルを展開するときに任意のファイルを上書きすることができる。これは悪用される危険があるので、<font color="red">信頼できない提供元のアーカイブを展開してはいけない</font>。

ファイル名の文字コードは規定されていない。このため、基本的にはファイルシステムが返すバイト列がそのまま格納される。非 ASCII 文字を含むファイル名は、異なる文字コード環境で展開すると「文字化け」が発生する可能性がある。

標準ライブラリの `tarfile` モジュールは、GZIP、BZIP2、LZMA、Zstandard 圧縮されたものを含む、TAR アーカイブを読み書きできる。ただし、Zstandard のサポートは Python 3.14 で追加された。

``` python
tarfile.is_tarfile(name)
```

この関数は、`name` が TAR 形式の場合 `True` を返し、そうでない場合 `False` を返す。`name` にはファイルのパス（文字列）かファイルオブジェクトを指定する。

``` python
tarfile.open(name=None, mode='r', fileobj=None, bufsize=10240, **kwargs)
```

この関数は、`tarfile.TarFile` オブジェクトを返す。主な引数は次のとおり。

| 引数 | 意味 |
|:---|:---|
| `name` | パス名。path-like オブジェクトを使用できる |
| `mode` | 読み込み: `'r'` / `'r:*'`（自動選択）、`'r:'`（非圧縮）、`'r:gz'`（GZIP）、`'r:bz2'`（BZIP2）、`'r:xz'`（LZMA2）、`'r:zst'`（Zstandard）<br /><br />書き出し: `'w'` / `'w:'`（非圧縮）、`'w:gz'`（GZIP）、`'w:bz2'`（BZIP2）、`'w:xz'`（LZMA2）<br /><br />新規作成: `'x'` / `'x:'`（非圧縮）、`'x:gz'`（GZIP）、`'x:bz2'`（BZIP2）、`'x:xz'`（LZMA2）<br /><br />追記: `'a'` / `'a:'`（自動選択） |
| `fileobj` | バイナリモードでオープンされたファイルオブジェクト。`name` と `fileobj` のいずれかを指定する |

開いた TAR ファイルは、処理が済んだら `close()` メソッドで閉じる必要がある。

`tarfile.TarFile` はコンテキストマネージャーになっているので、with 文をサポートする。`__enter__()` は自身を返し、`__exit__()` は `close()` を呼び出す。

`tarfile.TarFile` オブジェクトは、読み込み用に開かれている場合、イテラブルとして `tarfile.TarInfo` オブジェクトを 1 つずつ返す。

`tarfile.TarInfo` は、ヘッダー情報を格納する。主な属性は次のとおり。

| 属性 | 意味 |
|:---|:---|
| `TarInfo.name` | アーカイブ中のファイルの名前 |
| `TarInfo.mtime` | アーカイブ中のファイルの最終更新日時となるエポック秒 |
| `TarInfo.size` | アーカイブ中のファイルサイズ |
| `TarInfo.uid` | アーカイブ中のファイルを保存した元のユーザーの UID |
| `TarInfo.gid` | アーカイブ中のファイルを保存した元のユーザーの GID |
| `TarInfo.uname` | アーカイブ中のファイルを保存した元のユーザーのユーザー名 |
| `TarInfo.gname` | アーカイブ中のファイルを保存した元のユーザーのグループ名 |

■ `tarfile.TarFile` オブジェクトが読み込み用に開かれている場合に使用される、主なメソッドは次のとおり。

``` python
TarFile.getnames()
```

TAR ファイル内にアーカイブされているファイルの名前のリストを返す。

``` python
TarFile.extractall(path='.', members=None, *, numeric_owner=False, filter=None)
```

アーカイブ中の全てのファイルを展開する。

| 引数 | 意味 |
|:---|:---|
| `path` | 展開先のディレクトリ。デフォルトはカレントディレクトリ |
| `members` | イテラブルを指定した場合、イテラブルが返すメンバーだけを展開する |
| `numeric_owner` | キーワード専用。`False`（デフォルト）の場合、ヘッダーの `uname`/`gname` が展開先システムに存在すれば、そのユーザーに所有権を設定し、存<br /><br />在しなければヘッダーの `uid`/`gid` を適用する。`True` の場合、常にヘッダーの `uid`/`gid` を適用する |
| `filter` | キーワード専用。Python 3.12 で追加。TAR ファイルを展開して任意のファイルが上書きされないようにするための「展開フィルター」を設定する<br /><br />・`'fully_trusted'`: フィルターなし。アーカイブ内のメタデータをすべてそのまま信頼して展開先システムに適用する<br /><br />・`'tar'`: UNIX 系ファイルシステムに固有の TAR 機能の大部分を許可するが、明らかに悪意があるか予期しづらい機能はブロックする<br /><br />・`'data'`: UNIX 系ファイルシステム固有の機能の多くを無視またはブロックする<br /><br />`None`（デフォルト）の場合、`'fully_trusted'`（Python 3.14 で `'data'` に変更） |

Python 3.12 と Python 3.13 では `filter` が未指定の場合、`DeprecationWarning` 警告が表示される。

``` python
TarFile.extract(member, path='', set_attrs=True, *, numeric_owner=False, filter=None)
```

アーカイブ中の `member` だけを展開する。`member` はファイル名もしくは `tarfile.TarInfo` オブジェクト。`set_attrs` が `False` でない限り、ファイルの属性（所有者、最終更新時刻、モード）が設定される。他の引数は `extractall()` メソッドと同じ。

``` python
TarFile.extractfile(member)
```

アーカイブから `member` をファイルオブジェクトとして抽出する。`member` はファイル名か `tarfile.TarInfo` オブジェクトとする。

■ `tarfile.TarFile` オブジェクトが書き出し用に開かれている場合に使用される、主なメソッドは次のとおり。

``` python
TarFile.add(name, arcname=None, recursive=True, *, filter=None)
```

ファイルやディレクトリをアーカイブに追加する。

| 引数 | 意味 |
|:---|:---|
| `name` | アーカイブに追加するファイルやディレクトリのパス |
| `arcname` | これを指定すると、アーカイブ中ではその名前が使用される |
| `recursive` | `True`（デフォルト）の場合、`name` がディレクトリなら再帰的に追加される |
| `filter` | キーワード専用。`tarfile.TarInfo` オブジェクトを引数として受け取り、操作した `tarfile.TarInfo` オブジェクトを返す関数を指定すると、その<br /><br />戻り値をヘッダーに使用する。この関数が `None` を返す場合はアーカイブに追加されない |

TAR ファイルを作成する例:

In [10]:
import tarfile
import os

def reset(tarinfo):
    """ユーザー情報をリセットするフィルター関数"""
    tarinfo.uid = tarinfo.gid = 0
    tarinfo.uname = tarinfo.gname = "root"
    return tarinfo

# アーカイブを作成
with tarfile.open('example.tar.gz', mode='w:gz') as tar:
    tar.add('/content/sample_data/README.md', filter=reset)  # ファイルを追加
    tar.add('/content/sample_data/anscombe.json', filter=reset, arcname='sample.json')  # ファイル名を変更して追加

# 作成したアーカイブの確認
with tarfile.open('example.tar.gz') as tar:
    print(tar.getnames())

# アーカイブの一部を、ジェネレーター関数を利用して展開する
def py_files(members):
    for tarinfo in members:
        if os.path.splitext(tarinfo.name)[1] == '.json':
            yield tarinfo

with tarfile.open('example.tar.gz') as tar:
    tar.extractall(members=py_files(tar), filter='data')

['content/sample_data/README.md', 'sample.json']


### ZIP

**ZIP**（ジップ）は、PKWAREの PKZIP ユーティリティに実装された、アーカイブのフォーマットである。基本フォーマットがパブリックドメインとして公開され、誰でも ZIP ファイルを扱うアプリケーションを開発することができる。Windows は早い時期から ZIP の機能を組み込んでいる。

圧縮は ZIP アーカイブのオプションであり、必要に応じて各種ある圧縮アルゴリズムを選択・使用し、ファイルサイズを圧縮して格納することも可能である。また、暗号化にも対応している。

ZIP ファイルは一般的に .zip という拡張子が付けられる。ZIP フォーマットは、圧縮展開を主目的としない多くのアプリケーションでも使用されており、その際、拡張子にはアプリケーション固有の名前が用いられていることが多い。例えば、Microsoft Office の現在の標準ファイル形式は、拡張子が .docx、.xlsx などであるが、ZIP が使用されている。

ZIP フォーマットは、いくつかのファイルエントリとセントラルディレクトリから構成される（図は [Wikimedia Commons](https://commons.wikimedia.org/wiki/File:ZIPformat_ja.jpg) から引用）。

  1. **セントラルディレクトリ**:  
  ZIP フォーマットの最後尾にまとめて配置され、全体の「目次」として機能する。
      * **ファイルヘッダー**:  
      汎用目的のビットフラグ、ZIP ファイル内でエントリデータが位置するオフセット、ファイルサイズ、ファイル名、圧縮方式、最終変更日時、チェックサム（CRC32）、コメントなどのメタデータが格納される。
          * MS-DOS のファイルシステムである FAT に準拠していたため、65535 を超えるファイル数と 4GB を超えるファイルを扱えない。また、最終変更日時の範囲が「1980-01-01 00:00:00」から「2107-12-31 23:59:58」で、精度は 2 秒となる。
          * 可変長の領域である**拡張フィールド**が含まれる場合があり、ファイル数とファイルサイズの制約を解決するための ZIP64 形式、暗号化、NTFS や Unix ファイルのタイムスタンプと同じ精度、ファイル属性などをサポートするために使用される。
      * **終端レコード（EOCD）**:  
      先頭のシグネチャ `0x504B0506` とセントラルディレクトリのメタデータ
  2. **ファイルエントリ**
      * **ローカルファイルヘッダー**:  
      セントラルディレクトリ内のファイルヘッダーとほぼ同じメタデータが格納され、補助的な情報源として使用される。拡張フィールドが含まれる場合がある。
      * **ファイルデータ本体**:  
      圧縮されたり暗号化されている場合もある。

汎用目的のビットフラグを設定して UTF-8 でエンコードされたファイル名を格納することができる。

ヘッダーに格納される最終変更日時は、MS-DOS のファイルシステムである FAT 形式となり、範囲は「1980-01-01 00:00:00」から「2107-12-31 23:59:58」で、精度は 2 秒となる。

![](https://upload.wikimedia.org/wikipedia/commons/thumb/5/5e/ZIPformat_ja.jpg/960px-ZIPformat_ja.jpg)

Windows は、現在の ZIP の仕様を全てサポートしているわけではない。

  * 使用できる圧縮方式は無圧縮（Store）と Deflate のみである。
  * コメント機能とパスワード付き圧縮には全く対応していない。
  * パスワード付き ZIP の展開は可能だが、簡易な暗号方式にのみ対応している。この暗号化方式は簡単に解読可能なため、暗号化方式としては無意味である。
  * 圧縮時にはファイル名の文字コードが CP932 となり、UTF-8 エンコードに対応していない。展開時には CP932 と UTF-8 のどちらであっても問題なく展開できる。

標準ライブラリの `zipfile` パッケージは、ZIP 形式のファイルの作成、読み書き、追記、アーカイブ内のファイル一覧の作成を行うためのツールを提供している。

`zipfile` パッケージは、以下の圧縮方式に対応しており、数値定数で指定することができる。

| 圧縮方式 | 数値定数 | 圧縮レベル | 備考 |
|:---|:---|:---|:---|
| 非圧縮 | `zipfile.ZIP_STORED` | なし | |
| GZIP 圧縮方式 | `zipfile.ZIP_DEFLATED` | 0 から 9 まで（デフォルトは 6） | |
| BZIP2 圧縮方式 | `zipfile.ZIP_BZIP2` | 1 から 9 まで（デフォルトは 9） | |
| LZMA 圧縮方式 | `zipfile.ZIP_LZMA` |  なし | |
| Zstandard 圧縮方式 | `zipfile.ZIP_ZSTANDARD` | -1 から 22 まで（デフォルトは 3） | Python 3.14 で追加 |

`zipfile` パッケージが対応していない圧縮方式が使われている ZIP ファイルを読み取ろうとすると、`NotImplementedError` 例外が発生する。

また、正常ではない ZIP ファイルに対して、`zipfile.BadZipFile` 例外が送出される。

``` python
zipfile.is_zipfile(filename)
```

この関数は、`filename` が ZIP ファイルの場合 `True` を返し、そうでない場合 `False` を返す。`filename` にはファイルのパス（文字列）かファイルオブジェクトを指定する。

``` python
zipfile.ZipFile(file, mode='r', compression=ZIP_STORED, allowZip64=True, compresslevel=None, *, strict_timestamps=True,
                metadata_encoding=None)
```

`zipfile.ZipFile` は ZIP ファイルの読み書きのためのクラスである。コンストラクタの引数は次のとおり。

| 引数 | 意味 |
|:---|:---|
| `file` | ファイルのパス（文字列）か、ファイルオブジェクト、 path-like オブジェクト |
| `mode` | `'r'`（ZIP ファイルの読み込み）, `'w'`（新規 ZIP ファイルの作成）, `'a'`（ZIP ファイルへの追加）, `'x'`（ファイルが存在しない場<br /><br />合にのみ ZIP ファイルを作成して書き込む）を指定できる |
| `compression` | デフォルトで使用される圧縮方式を数値で指定する。デフォルトでは非圧縮とされることに注意する |
| `allowZip64` | `True`（デフォルト）の場合、ZIP64 形式に対応する |
| `compresslevel` | デフォルトで使用される圧縮レベルを指定する |
| `strict_timestamps` | キーワード専用。`False` に設定すると、「1980-01-01」以前や「2107-12-31」以後のタイムスタンプがあってもエラーにならず、タ<br /><br />イムスタンプはそれぞれ下限値と上限値に設定される |
| `metadata_encoding` | キーワード専用。Python 3.11 で追加。非 ASCII 文字を含むファイル名が文字化けするなら、Unix 系 OS では `'utf-8'` を指定し、<br /><br />Windows では `'cp932'` を指定すると解決する |

`zipfile.ZipFile` の主なメソッドは以下のとおり。

``` python
ZipFile.namelist()
```

ZIP ファイルに含まれる各ファイルの名前のリストを返す。

``` python
ZipFile.infolist()
```

ZIP ファイルに含まれる各ファイルに関する `zipfile.ZipInfo` オブジェクトからなるリストを返す。

`zipfile.ZipInfo` クラスは、ZIP のファイルヘッダー内メタデータを格納しており、次のような属性を持つ。

| 属性 | 意味 |
|:---|:---|
| `ZipInfo.filename` | アーカイブ中のファイルの名前 |
| `ZipInfo.date_time` | アーカイブ中のファイルの最終更新日時。西暦年（>= 1980）、月、日、時、分、秒を示す 6 要素タプル |
| `ZipInfo.compress_size` | 圧縮後のデータのサイズ |
| `ZipInfo.file_size` | 圧縮前のファイルのサイズ |

``` python
ZipFile.extractall(path=None, members=None, pwd=None)
```

ZIP ファイルに含まれるファイルを全て展開する。`path` には、展開先の別のディレクトリを指定する。`members` にファイルの名前のリストを指定した場合、`members` だけを展開する。`pwd` は暗号化された ZIP ファイルに使用されるパスワードを指定する。

``` python
ZipFile.extract(member, path=None, pwd=None)
```

ZIP ファイルに含まれるファイルを指定して展開する。`member` にはファイルの名前または `zipfile.ZipInfo` オブジェクトを指定する。

``` python
ZipFile.write(filename, arcname=None, compress_type=None, compresslevel=None)
```

指定したファイルを ZIP ファイルにアーカイブする。`arcname` を指定するとその名前でアーカイブする。`filename` と `arcname` はアーカイブルートに対する相対パスでなければならない。`compress_type` で圧縮方式を指定できる。`compresslevel` で圧縮レベルを指定できる。

``` python
ZipFile.close()
```

ZIP ファイルを閉じる。

`zipfile.ZipFile` はコンテキストマネージャーにもなっているので、with 文をサポートする。`__enter__()` は自身を返し、`__exit__()` は `close()` を呼び出す。

In [4]:
import zipfile

# アーカイブを作成
with zipfile.ZipFile('example.zip', 'w', zipfile.ZIP_DEFLATED) as zip:
    zip.write('sample_data/README.md', arcname='README.md')
    zip.write('sample_data/anscombe.json')

# 作成したアーカイブの確認
zip.namelist()

# アーカイブの一部を展開
with zipfile.ZipFile('example.zip') as zip:
    zip.extract('README.md')

### フォルダ単位のアーカイブ操作

`shutil` モジュールは、フォルダを丸ごとアーカイブ化するための高レベル API を提供している。

``` python
shutil.make_archive(base_name, format[, root_dir[, base_dir[, verbose[, dry_run[, owner[, group[, logger]]]]]]])
```

この関数は、アーカイブファイルを作成してその名前を返す。主な引数は、次のとおり。

| 引数 | 意味 |
|:---|:---|
| `base_name` | 作成するアーカイブファイルの名前を指定する。拡張子は指定しないこと |
| `format` | アーカイブ方式を指定する。`'zip'`, `'tar'`, `'gztar'`, `'bztar'`, `'xztar'`, `'zstdtar'`（Python 3.14 で追加）を指定できる。`'zip'` の場合、<br /><br />GZIP 圧縮方式のデフォルトレベルが使用される |
| `root_dir` | アーカイブ対象となるディレクトリを指定する。デフォルトはカレントディレクトリ |
| `base_dir` | アーカイブを開始するディレクトリを、`root_dir` からの相対パスで指定する。デフォルトはカレントディレクトリ |

アーカイブに含まれる全てのパスは `root_dir` からの相対パスになる。

`root_dir` を指定して `base_dir` を指定しない場合、`root_dir` そのものではなくその中身が圧縮対象になる。たとえば、`root_dir=target_dir` とする場合、圧縮前と圧縮後のディレクトリ構成は次のように対応する。

``` text
（圧縮前）
current_dir
└target_dir --- root_dir
  ├─ sub_dir
  │   ├─ a.txt
  │   └─ b.txt
  ├─ c.txt
  └─ d.txt

（圧縮後）
sub_dir
├─ a.txt
└─ b.txt
c.txt
d.txt
```

`root_dir` と `base_dir` の両方を指定する場合、`base_dir` そのものとその中身が圧縮対象になる。たとえば、`root_dir='.', base_dir='target_dir'` とする場合、圧縮前と圧縮後のディレクトリ構成は次のように対応する。

``` text
（圧縮前）
current_dir --- root_dir
└target_dir --- base_dir
  ├─ sub_dir
  │   ├─ a.txt
  │   └─ b.txt
  ├─ c.txt
  └─ d.txt

（圧縮後）
target_dir
├─ sub_dir
│   ├─ a.txt
│   └─ b.txt
├─ c.txt
└─ d.txt
```

``` python
shutil.unpack_archive(filename[, extract_dir[, format[, filter]]])
```

この関数は、アーカイブをアンパックする。主な引数は次のとおり。

| 引数 | 意味 |
|:---|:---|
| `filename` | アーカイブの絶対パスを path-like オブジェクトとして指定する |
| `extract_dir` | アーカイブをアンパックする先のディレクトリ名を path-like オブジェクトとして指定する。指定されなかった場合はカレントディレクトリが使われ<br /><br />る |
| `format` | `filename` のアーカイブ形式を指定する。`zip`, `tar`, `gztar`, `bztar`, `xztar` を指定できる。指定を省略した場合、`filename` の拡張子から<br /><br />推測される |